In [16]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from pprint import pprint

# Carregamento dos dados
dados = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Appendicitis/app_data - All cases.csv", encoding="ISO-8859-1")

# Guardar as variáveis-alvo separadamente
variaveis_alvo = ["Diagnosis", "Severity", "Management"]
dados_alvos = dados[variaveis_alvo].copy()

# Pré-processamento leve
for col in dados.columns:
    if dados[col].dtype == "object" and col not in variaveis_alvo:
        dados[col] = dados[col].str.replace(",", ".", regex=False)
        try:
            dados[col] = dados[col].astype(float)
        except:
            pass

# Preencher valores ausentes
dados = dados.fillna(dados.mean(numeric_only=True))

# Codificar variáveis-alvo com LabelEncoder
alvos_codificados = {}
for alvo in variaveis_alvo:
    le = LabelEncoder()
    dados_alvos[alvo] = le.fit_transform(dados_alvos[alvo])
    alvos_codificados[alvo] = le

# Remover as colunas-alvo antes de dummificar
dados_sem_alvo = dados.drop(columns=variaveis_alvo)
dados_dummies = pd.get_dummies(dados_sem_alvo)

# Juntar os dados dummificados com os alvos codificados
dados_final = pd.concat([dados_dummies, dados_alvos], axis=1)

# Função para treinar modelo para cada variável-alvo usando RandomOverSampler
def treinar_modelo_para_alvo(alvo):
    print(f"\n======= Variável alvo: {alvo} =======")
    dados_copia = dados_final.copy()

    if alvo not in dados_copia.columns:
        raise ValueError(f"Coluna '{alvo}' não encontrada no DataFrame.")

    # Separar atributos e classes
    dados_atributos = dados_copia.drop(columns=[alvo])
    dados_classes = dados_copia[alvo]

    # Verificar balanceamento original
    print("Distribuição original:", Counter(dados_classes))

    # Balancear com RandomOverSampler (sem necessidade de k_neighbors)
    ros = RandomOverSampler(random_state=42)
    atributos_b, classes_b = ros.fit_resample(dados_atributos, dados_classes)

    print("Distribuição após RandomOverSampler:", Counter(classes_b))

    # Treinar e avaliar modelo com Hold-Out
    attr_train, attr_test, class_train, class_test = train_test_split(
        atributos_b, classes_b, test_size=0.3, random_state=42
    )
    tree = DecisionTreeClassifier(random_state=42)
    tree.fit(attr_train, class_train)
    predictions = tree.predict(attr_test)
    print("Acurácia com Hold-Out:", accuracy_score(class_test, predictions))

    # Cross-Validation
    scores = cross_val_score(tree, atributos_b, classes_b, cv=10)
    print("Acurácia com Cross-Validation:", scores.mean())

    # Otimização de hiperparâmetros
    grid = {
        "criterion": ["gini", "entropy", "log_loss"],
        "splitter": ["best", "random"],
        "max_depth": [3, 6, 9],
        "max_features": ["sqrt", "log2"],
    }

    search = RandomizedSearchCV(tree, grid, cv=10, random_state=42, n_jobs=-1)
    search.fit(atributos_b, classes_b)

    print("Melhores hiperparâmetros:")
    pprint(search.best_params_)

    # Treinar modelo final com melhores parâmetros
    modelo_final = DecisionTreeClassifier(**search.best_params_)
    modelo_final.fit(atributos_b, classes_b)

    return modelo_final

# Treinar modelos para cada variável-alvo
modelo_diagnosis = treinar_modelo_para_alvo("Diagnosis")
modelo_severity = treinar_modelo_para_alvo("Severity")
modelo_management = treinar_modelo_para_alvo("Management")



======= Variável alvo: Diagnosis =======
Distribuição original: Counter({0: 463, 1: 317, 2: 2})
Distribuição após RandomOverSampler: Counter({0: 463, 1: 463, 2: 463})
Acurácia com Hold-Out: 0.9784172661870504
Acurácia com Cross-Validation: 0.9741007194244606
Melhores hiperparâmetros:
{'criterion': 'entropy',
 'max_depth': 9,
 'max_features': 'sqrt',
 'splitter': 'random'}

======= Variável alvo: Severity =======
Distribuição original: Counter({1: 662, 0: 119, 2: 1})
Distribuição após RandomOverSampler: Counter({1: 662, 2: 662, 0: 662})
Acurácia com Hold-Out: 0.9848993288590604
Acurácia com Cross-Validation: 0.9732373991167961
Melhores hiperparâmetros:
{'criterion': 'gini',
 'max_depth': 9,
 'max_features': 'sqrt',
 'splitter': 'best'}

======= Variável alvo: Management =======
Distribuição original: Counter({0: 483, 1: 270, 2: 27, 4: 1, 3: 1})
Distribuição após RandomOverSampler: Counter({0: 483, 4: 483, 1: 483, 2: 483, 3: 483})
Acurácia com Hold-Out: 0.9751724137931035
Acurácia com C